<a href="https://colab.research.google.com/github/Johoodcoder/CS490Project/blob/whittington/Notebooks/CS490Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Non-preinstalled module installs

In [1]:
!pip install pytorch-pretrained-bert
!pip install pytorch-nlp

     |████████████████████████████████| 133kB 8.0MB/s 
     |████████████████████████████████| 133kB 13.8MB/s 
     |████████████████████████████████| 7.3MB 10.1MB/s 
     |████████████████████████████████| 81kB 11.8MB/s 
ERROR: botocore 1.20.39 has requirement urllib3<1.27,>=1.25.4, but you'll have urllib3 1.24.3 which is incompatible.
     |████████████████████████████████| 92kB 5.4MB/s 


Import Dataset used in https://towardsdatascience.com/fake-news-classification-with-bert-afbeee601f41

In [ ]:
# from google.colab import files

# uploaded = files.upload()

# for fn in uploaded.keys():
#   print('User uploaded file "{name}" with length {length} bytes'.format(
#       name=fn, length=len(uploaded[fn])))

The base code from https://github.com/spierre91/medium_code/blob/master/fake_news_classifcation.py

In [2]:
import pandas as pd 
import numpy as np 
import torch.nn as nn
from pytorch_pretrained_bert import BertTokenizer, BertModel
import torch
from torchnlp.datasets import imdb_dataset
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report
import gc

In [3]:
pd.set_option('display.max_columns', None)
train_data, test_data = imdb_dataset(train=True, test=True)
#df = pd.read_csv("condensed_fake_real_news_SANITIZED.csv")
df = pd.read_csv("fnn_train.csv")
#df = df[['text', 'type']]
df = df[['fullText_based_content', 'label_fnn']]
print(len(df))

aclImdb_v1.tar.gz: 84.1MB [00:01, 48.2MB/s]                            


15212


In [4]:
from collections import Counter 

#print(Counter(df['type'].values))
print(Counter(df['label_fnn'].values))

Counter({'fake': 7621, 'real': 7591})


In [5]:
df = df[df['label_fnn'].isin(['fake', 'real'])]
#df = df[df['type'].isin(['fake', 'real'])]
df.dropna(inplace = True)
df = df.sample(frac=1, random_state = 24).reset_index(drop=True)

#print(Counter(df['type'].values))
print(Counter(df['label_fnn'].values))

Counter({'fake': 7621, 'real': 7591})


In [6]:
train_data_df = df.head(3200)
test_data_df = df.tail(800)
print(train_data_df)

                                 fullText_based_content label_fnn
0     The country's economic woes were a somber topi...      real
1     The book "End of Days" by Sylvia Browne explor...      fake
2     Partisan narratives that emerged after the dea...      fake
3     The president’s political opponents have "been...      fake
4     In the final days before the election, the Oba...      real
...                                                 ...       ...
3195  Rumors and news reports citing anonymous sourc...      fake
3196  Democratic presidential candidate Hillary Clin...      real
3197  Cormick Lynch, a Republican running to replace...      real
3198  In a U.S. population of 329 million, is it pos...      fake
3199  A political flier directed to voters in north ...      fake

[3200 rows x 2 columns]


In [7]:
# train_data = []
# for index, row in train_data_df.iterrows():
#     train_data.append({'text': row['text'], 'type': row['type']})

# test_data = []
# for index, row in test_data_df.iterrows():
#     test_data.append({'text': row['text'], 'type': row['type']})

train_data = []
for index, row in train_data_df.iterrows():
    train_data.append({'fullText_based_content': row['fullText_based_content'], 'label_fnn': row['label_fnn']})

test_data = []
for index, row in test_data_df.iterrows():
    test_data.append({'fullText_based_content': row['fullText_based_content'], 'label_fnn': row['label_fnn']})


# train_data = [{'text': text, 'type': type_data } for text in list(train_data['text']) for type_data in list(train_data['type'])]
# test_data = [{'text': text, 'type': type_data } for text in list(test_data['text']) for type_data in list(test_data['type'])]
# #gc.collect()

In [8]:
# train_texts, train_labels = list(zip(*map(lambda d: (d['text'], d['type']), train_data)))
# test_texts, test_labels = list(zip(*map(lambda d: (d['text'], d['type']), test_data)))

train_texts, train_labels = list(zip(*map(lambda d: (d['fullText_based_content'], d['label_fnn']), train_data)))
test_texts, test_labels = list(zip(*map(lambda d: (d['fullText_based_content'], d['label_fnn']), test_data)))

In [9]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], train_texts))
test_tokens = list(map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:511], test_texts))

train_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, train_tokens))
test_tokens_ids = list(map(tokenizer.convert_tokens_to_ids, test_tokens))



train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=512, truncating="post", padding="post", dtype="int")


100%|██████████| 231508/231508 [00:00<00:00, 1201597.98B/s]


In [10]:
train_y = np.array(train_labels) == 'fake'
test_y = np.array(test_labels) == 'fake'

In [11]:
print(train_y)
print(test_y)

[False  True  True ... False  True  True]
[ True False False  True  True False False  True  True  True False False
  True False False  True False  True False  True False False  True False
 False  True  True  True False False False  True False False False False
  True  True False False  True False  True  True False False False False
  True  True False  True  True  True  True False False False  True  True
 False  True  True False  True False  True False False False  True False
  True  True  True False False  True False  True False False  True False
  True False False  True False  True False False False False  True False
 False False False False  True  True False  True  True False  True False
  True  True  True False False False False  True  True False False False
  True False False False  True False False False  True  True False  True
 False  True  True False  True False  True  True False  True  True False
 False False False  True False False  True False False False False  True
  True Fa

In [12]:
class BertBinaryClassifier(nn.Module):
    def __init__(self, dropout=0.1):
        super(BertBinaryClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-uncased')

        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, tokens, masks=None):
        _, pooled_output = self.bert(tokens, attention_mask=masks, output_all_encoded_layers=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        proba = self.sigmoid(linear_output)
        return proba

In [13]:
train_masks = [[float(i > 0) for i in ii] for ii in train_tokens_ids]
test_masks = [[float(i > 0) for i in ii] for ii in test_tokens_ids]
train_masks_tensor = torch.tensor(train_masks)
test_masks_tensor = torch.tensor(test_masks)

In [14]:
train_tokens_tensor = torch.tensor(train_tokens_ids)
train_y_tensor = torch.tensor(train_y.reshape(-1, 1)).float()
test_tokens_tensor = torch.tensor(test_tokens_ids)
test_y_tensor = torch.tensor(test_y.reshape(-1, 1)).float()

train_masks_tensor = train_masks_tensor.to('cuda')
test_masks_tensor = test_masks_tensor.to('cuda')

train_tokens_tensor = train_tokens_tensor.to('cuda')
test_tokens_tensor = test_tokens_tensor.to('cuda')
train_y_tensor = train_y_tensor.to('cuda')
test_y_tensor = test_y_tensor.to('cuda')

In [15]:
BATCH_SIZE = 8
EPOCHS = 4

In [16]:
train_dataset =  torch.utils.data.TensorDataset(train_tokens_tensor, train_masks_tensor, train_y_tensor)
train_sampler =  torch.utils.data.RandomSampler(train_dataset)
train_dataloader =  torch.utils.data.DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)
test_dataset =  torch.utils.data.TensorDataset(test_tokens_tensor, test_masks_tensor, test_y_tensor)
test_sampler =  torch.utils.data.SequentialSampler(test_dataset)
test_dataloader =  torch.utils.data.DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [17]:
bert_clf = BertBinaryClassifier()
bert_clf.to('cuda')
optimizer = torch.optim.Adam(bert_clf.parameters(), lr=3e-5)
for epoch_num in range(EPOCHS):
    bert_clf.train()
    train_loss = 0
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, masks, labels = tuple(t for t in batch_data)
        probas = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        batch_loss = loss_func(probas, labels)
        train_loss += batch_loss.item()
        bert_clf.zero_grad()
        batch_loss.backward()
        optimizer.step()
        print('Epoch: ', epoch_num + 1)
        print("\r" + "{0}/{1} loss: {2} ".format(step_num, len(train_data) / BATCH_SIZE, train_loss / (step_num + 1)))

100%|██████████| 407873900/407873900 [00:10<00:00, 38742818.96B/s]


Epoch:  1
0/400.0 loss: 0.6798498630523682 
Epoch:  1
1/400.0 loss: 0.712151825428009 
Epoch:  1
2/400.0 loss: 0.6844405730565389 
Epoch:  1
3/400.0 loss: 0.7049208283424377 
Epoch:  1
4/400.0 loss: 0.7287379026412963 
Epoch:  1
5/400.0 loss: 0.7207578718662262 
Epoch:  1
6/400.0 loss: 0.7159675444875445 
Epoch:  1
7/400.0 loss: 0.7181136086583138 
Epoch:  1
8/400.0 loss: 0.7069890830251906 
Epoch:  1
9/400.0 loss: 0.7052681267261505 
Epoch:  1
10/400.0 loss: 0.7063016024502841 
Epoch:  1
11/400.0 loss: 0.7074139912923177 
Epoch:  1
12/400.0 loss: 0.7118429037240835 
Epoch:  1
13/400.0 loss: 0.7086231367928642 
Epoch:  1
14/400.0 loss: 0.7090950608253479 
Epoch:  1
15/400.0 loss: 0.7058736868202686 
Epoch:  1
16/400.0 loss: 0.7036453029688667 
Epoch:  1
17/400.0 loss: 0.7007512052853903 
Epoch:  1
18/400.0 loss: 0.6989729404449463 
Epoch:  1
19/400.0 loss: 0.6994865715503693 
Epoch:  1
20/400.0 loss: 0.6983175902139573 
Epoch:  1
21/400.0 loss: 0.6982231465252963 
Epoch:  1
22/400.0 lo

In [ ]:
bert_clf.eval()
bert_predicted = []
all_logits = []
with torch.no_grad():
    for step_num, batch_data in enumerate(test_dataloader):

        token_ids, masks, labels = tuple(t for t in batch_data)

        logits = bert_clf(token_ids, masks)
        loss_func = nn.BCELoss()
        loss = loss_func(logits, labels)
        numpy_logits = logits.cpu().detach().numpy()
        
        bert_predicted += list(numpy_logits[:, 0] > 0.5)
        all_logits += list(numpy_logits[:, 0])
        
print(classification_report(test_y, bert_predicted))


              precision    recall  f1-score   support

       False       0.63      0.76      0.69       429
        True       0.63      0.47      0.54       371

    accuracy                           0.63       800
   macro avg       0.63      0.62      0.62       800
weighted avg       0.63      0.63      0.62       800



In [ ]:
print(test_y)
print(bert_predicted)

[ True False False  True  True False False  True  True  True False False
  True False False  True False  True False  True False False  True False
 False  True  True  True False False False  True False False False False
  True  True False False  True False  True  True False False False False
  True  True False  True  True  True  True False False False  True  True
 False  True  True False  True False  True False False False  True False
  True  True  True False False  True False  True False False  True False
  True False False  True False  True False False False False  True False
 False False False False  True  True False  True  True False  True False
  True  True  True False False False False  True  True False False False
  True False False False  True False False False  True  True False  True
 False  True  True False  True False  True  True False  True  True False
 False False False  True False False  True False False False False  True
  True False  True False  True  True False False  T